# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

!pip install keras #only needed when running from tensorflow docker container

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
import keras
import tensorflow as tf
print("keras version: {}".format(keras.__version__))
print("tensorflow version: {}".format(tf.__version__))
print(device_lib.list_local_devices())

keras version: 2.4.3
tensorflow version: 2.3.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10730224504940025092
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7040116858334987708
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3734509023726442669
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10144673984
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14760785701647995299
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:07:00.0, compute capability: 6.1"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
print("total sentences: {}".format(len(english_sentences)))

for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

total sentences: 137861
small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

english_words_set = set()

for sentence in english_sentences:
    for word in sentence.split():
        english_words_set.add(word)
print ("\n{} unique english words using a normal loop.".format(len(english_words_set)))

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"

227 unique english words using a normal loop.


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    
    #return None, None
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    padded_sentences = pad_sequences(x, maxlen=length, padding='post') 
    return padded_sentences

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

def tokens_to_text(tokens, tokenizer):
    """
    Turn tokens into words
    :param tokens: numpy array of tokens.  shape (1,length,1)
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the tokens
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return_text = ""
    
    for token in tokens[0]:
        if isinstance(token, np.ndarray):
            return_text += index_to_words[token[0]]
        else:
            return_text += index_to_words[token]
        return_text += ' '
    
    return return_text

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    print("input_shape: {}".format(input_shape))
    print("output_sequence_length: {}".format(output_sequence_length))
    print("english_vocab_size: {}".format(english_vocab_size))
    print("french_vocab_size: {}".format(french_vocab_size))
    print("\n")

    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    
    learning_rate = 1e-3 
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

input_shape: (137861, 21, 1)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


input_shape: (137861, 21, 1)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


Epoch 1/10
108/108 [==============================] - 2s 16ms/step - loss: 3.4749 - accuracy: 0.4167 - val_loss: nan - val_accuracy: 0.4606
Epoch 2/10
108/108 [==============================] - 1s 13ms/step - loss: 2.4429 - accuracy: 0.4684 - val_loss: nan - val_accuracy: 0.4792
Epoch 3/10
108/108 [==============================] - 1s 13ms/step - loss: 2.2106 - accuracy: 0.5176 - val_loss: nan - val_accuracy: 0.5428
Epoch 4/10
108/108 [==============================] - 1s 13ms/step - loss: 1.9780 - accuracy: 0.5566 - val_loss: nan - val_accuracy: 0.5656
Epoch 5/10
108/108 [==============================] - 1s 13ms/step - loss: 1.8046 - accuracy: 0.5713 - val_loss: nan - val_accuracy: 0.5778
Epoch 6/10
108/108 [==============================] - 1s 13ms/step - loss: 1.7033 - accura

In [13]:
# Print prediction(s)
for i in range(10):
    print("english:", tokens_to_text(tmp_x[i:i+1], english_tokenizer).replace('<PAD>', ''))
    prediction = simple_rnn_model.predict(tmp_x[i:i+1])
    print("french: ", logits_to_text(prediction[0], french_tokenizer).replace('<PAD>', ''))
    print("\n")

english: new jersey is sometimes quiet during autumn and it is snowy in april         
french:  new jersey est parfois parfois en en mais il est est en en        


english: the united states is usually chilly during july and it is usually freezing in november       
french:  la états unis est jamais chaud en en mais il est est agréable en en      


english: california is usually quiet during march and it is usually hot in june         
french:  new est est jamais en en et il est est est en en        


english: the united states is sometimes mild during june and it is cold in september        
french:  la états unis est parfois parfois en en mais il est est en en       


english: your least liked fruit is the grape but my least liked is the apple        
french:  elle fruit le moins aimé la la mais son moins aimé aimé la la       


english: his favorite fruit is the orange but my favorite is the grape          
french:  elle fruit préféré est la la mais son préféré est la la       

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [14]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print("input_shape: {}".format(input_shape))
    print("output_sequence_length: {}".format(output_sequence_length))
    print("english_vocab_size: {}".format(english_vocab_size))
    print("french_vocab_size: {}".format(french_vocab_size))

    input_seq = Input(input_shape[1:])
    print("input_seq.shape: {}".format(input_seq.shape))
    embedding = Embedding(english_vocab_size+1, 64, input_length=input_shape[1])(input_seq)
    print("embedding.shape: {}".format(embedding.shape))
    print("\n")
    rnn = GRU(64, return_sequences=True)(embedding)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    
    learning_rate = 1e-3 
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# TODO: Train the neural network
simple_embed_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_embed_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_embed_model.predict(tmp_x[:1])[0], french_tokenizer))

input_shape: (137861, 21)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344
input_seq.shape: (None, 21)
embedding.shape: (None, 21, 64)


input_shape: (137861, 21)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344
input_seq.shape: (None, 21)
embedding.shape: (None, 21, 64)


Epoch 1/10
108/108 [==============================] - 2s 18ms/step - loss: 3.7481 - accuracy: 0.4011 - val_loss: nan - val_accuracy: 0.4093
Epoch 2/10
108/108 [==============================] - 2s 15ms/step - loss: 2.6350 - accuracy: 0.4574 - val_loss: nan - val_accuracy: 0.5206
Epoch 3/10
108/108 [==============================] - 2s 15ms/step - loss: 1.9185 - accuracy: 0.5683 - val_loss: nan - val_accuracy: 0.6172
Epoch 4/10
108/108 [==============================] - 2s 16ms/step - loss: 1.4235 - accuracy: 0.6547 - val_loss: nan - val_accuracy: 0.6950
Epoch 5/10
108/108 [==============================] - 2s 15ms/step - loss: 1.1110 - accuracy: 0.7274 - val_loss: nan

In [17]:
# Print prediction(s)
for i in range(10):
    print("english: ", tokens_to_text(tmp_x[i:i+1], english_tokenizer).replace('<PAD>', ''))
    prediction = simple_embed_model.predict(tmp_x[i:i+1])
    print("french:  ", logits_to_text(prediction[0], french_tokenizer).replace('<PAD>', ''))
    print("\n")

english:  new jersey is sometimes quiet during autumn and it is snowy in april         
french:   new jersey est parfois calme en l' et il il neigeux neigeux en avril       


english:  the united states is usually chilly during july and it is usually freezing in november       
french:   les états unis est généralement froid en juillet et il est généralement en en novembre      


english:  california is usually quiet during march and it is usually hot in june         
french:   californie est généralement calme en mars et il est est chaud chaud juin        


english:  the united states is sometimes mild during june and it is cold in september        
french:   les états unis est parfois doux en juin et il est froid en septembre       


english:  your least liked fruit is the grape but my least liked is the apple        
french:   votre fruit moins moins aimé la raisin mais mon moins aimé est la pomme       


english:  his favorite fruit is the orange but my favorite is the grape  

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [35]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print("input_shape: {}".format(input_shape))
    print("output_sequence_length: {}".format(output_sequence_length))
    print("english_vocab_size: {}".format(english_vocab_size))
    print("french_vocab_size: {}".format(french_vocab_size))
    print("\n")
    
    input_seq = Input(input_shape[1:])
    
    input_seq_squeezed = keras.backend.squeeze(input_seq, 2)
    embedding = Embedding(english_vocab_size+1, 64, input_length=input_shape[1])(input_seq_squeezed)
    rnn = Bidirectional(GRU(64, return_sequences=True))(embedding)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    
    learning_rate = 1e-3 
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    #print(model.summary())
    
    return model

tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)
# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_bd_model.predict(tmp_x[:1])[0], french_tokenizer))

input_shape: (137861, 21, 1)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344
input_seq.shape: (None, 21, 1)
embedding.shape: (None, 21, 64)


Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 21, 1)]           0         
_________________________________________________________________
tf_op_layer_Squeeze_4 (Tenso [(None, 21)]              0         
_________________________________________________________________
embedding_19 (Embedding)     (None, 21, 64)            12800     
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 21, 128)           49920     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 21, 344)           44376     
_________________________________________________________________
activation_7 (Act

In [36]:
for i in range(20):
    print(tokens_to_text(tmp_x[i:i+1], english_tokenizer).replace('<PAD>', ''))
    prediction = simple_bd_model.predict(tmp_x[i:i+1])
    print(logits_to_text(prediction[0], french_tokenizer).replace('<PAD>', ''))
    print("\n")

new jersey is sometimes quiet during autumn and it is snowy in april         
new jersey est parfois calme en automne et il il neigeux en en avril       


the united states is usually chilly during july and it is usually freezing in november       
les états unis est généralement froid en juillet et il est habituellement en en novembre      


california is usually quiet during march and it is usually hot in june         
californie est généralement calme en mars et il est généralement chaud en juin        


the united states is sometimes mild during june and it is cold in september        
les états unis est parfois doux en juin et il est froid en septembre       


your least liked fruit is the grape but my least liked is the apple        
votre fruit moins moins est le raisin mais mon moins aimé est la pomme       


his favorite fruit is the orange but my favorite is the grape          
son fruit préféré est l'orange mais mais préféré préféré est           


paris is relaxing du

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [38]:

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    print("input_shape: {}".format(input_shape))
    print("output_sequence_length: {}".format(output_sequence_length))
    print("english_vocab_size: {}".format(english_vocab_size))
    print("french_vocab_size: {}".format(french_vocab_size))
    print("\n")
    gru_units = 256
    fc_units  = 256
    
    input_seq = Input(input_shape[1:])
    
    encoder_rnn = GRU(gru_units, return_sequences=False)(input_seq)
    #print("encoder_rnn.shape: {}".format(encoder_rnn.shape))
    
    repeat_vector = RepeatVector(output_sequence_length)(encoder_rnn)
    #print("repeat_vector.shape: {}".format(repeat_vector.shape))
    
    decoder_rnn = GRU(gru_units, return_sequences=True)(repeat_vector)
    #print("decoder_rnn.shape: {}".format(decoder_rnn.shape))
    
    fc_layer = Dense(fc_units, activation='relu')(decoder_rnn)
    
    logits = TimeDistributed(Dense(french_vocab_size))(fc_layer)

    model = Model(input_seq, Activation('softmax')(logits))
    
    learning_rate = 1e-3 
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)
# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_encdec_model.predict(tmp_x[:1])[0], french_tokenizer))



input_shape: (137861, 15, 1)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


input_shape: (137861, 21, 1)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


Epoch 1/10
862/862 [==============================] - 7s 9ms/step - loss: 1.8117 - accuracy: 0.5567 - val_loss: nan - val_accuracy: 0.6156
Epoch 2/10
862/862 [==============================] - 7s 8ms/step - loss: 1.2903 - accuracy: 0.6313 - val_loss: nan - val_accuracy: 0.6509
Epoch 3/10
862/862 [==============================] - 7s 8ms/step - loss: 1.1801 - accuracy: 0.6583 - val_loss: nan - val_accuracy: 0.6632
Epoch 4/10
862/862 [==============================] - 7s 8ms/step - loss: 1.1447 - accuracy: 0.6675 - val_loss: nan - val_accuracy: 0.6728
Epoch 5/10
862/862 [==============================] - 7s 8ms/step - loss: 1.0818 - accuracy: 0.6786 - val_loss: nan - val_accuracy: 0.6912
Epoch 6/10
862/862 [==============================] - 7s 8ms/step - loss: 0.9845 - accuracy: 0.

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [39]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print("input_shape: {}".format(input_shape))
    print("output_sequence_length: {}".format(output_sequence_length))
    print("english_vocab_size: {}".format(english_vocab_size))
    print("french_vocab_size: {}".format(french_vocab_size))
    print("\n")
    learning_rate = 1e-3 
    embedding_size = 256
    gru_units = 256
    fc_units  = 256
    
    input_seq = Input(input_shape[1:])
    
    print("input_seq.shape: {}".format(input_seq.shape))
    embedding = Embedding(english_vocab_size+1, embedding_size, input_length=input_shape[1])(input_seq)
    print("embedding.shape: {}".format(embedding.shape))
    
    #encoder_rnn = GRU(gru_units, return_sequences=False)(embedding)
    #print("encoder_rnn.shape: {}".format(encoder_rnn.shape))
    
    forward_encoder_layer  = GRU(gru_units, return_sequences=False)
    backward_encoder_layer = GRU(gru_units, return_sequences=False, go_backwards=True)
    encoder_rnn = Bidirectional(forward_encoder_layer, backward_layer=backward_encoder_layer)(embedding)
    
    repeat_vector = RepeatVector(output_sequence_length)(encoder_rnn)
    #print("repeat_vector.shape: {}".format(repeat_vector.shape))
    
    #decoder_rnn = GRU(gru_units, return_sequences=True)(repeat_vector)
    
    forward_decoder_layer  = GRU(gru_units, return_sequences=True)
    backward_decoder_layer = GRU(gru_units, return_sequences=True, go_backwards=True)
    decoder_rnn = Bidirectional(forward_decoder_layer, backward_layer=backward_decoder_layer)(repeat_vector)
    
    #print("decoder_rnn.shape: {}".format(decoder_rnn.shape))
    
    fc_layer = Dense(fc_units, activation='relu')(decoder_rnn)
    
    logits = TimeDistributed(Dense(french_vocab_size))(decoder_rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')
# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

input_shape: (137861, 15)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


input_seq.shape: (None, 15)
embedding.shape: (None, 15, 256)
Final Model Loaded
input_shape: (137861, 21)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


input_seq.shape: (None, 21)
embedding.shape: (None, 21, 256)
Epoch 1/10
862/862 [==============================] - 15s 17ms/step - loss: 1.3114 - accuracy: 0.6660 - val_loss: nan - val_accuracy: 0.7635
Epoch 2/10
862/862 [==============================] - 14s 16ms/step - loss: 0.5805 - accuracy: 0.8281 - val_loss: nan - val_accuracy: 0.8765
Epoch 3/10
862/862 [==============================] - 14s 16ms/step - loss: 0.2744 - accuracy: 0.9203 - val_loss: nan - val_accuracy: 0.9361
Epoch 4/10
862/862 [==============================] - 14s 16ms/step - loss: 0.1654 - accuracy: 0.9504 - val_loss: nan - val_accuracy: 0.9564
Epoch 5/10
862/862 [==============================] - 14s 16ms/step - loss: 0.1231 - accura

In [44]:
for i in range(200):
    print("english: ", tokens_to_text(tmp_x[i:i+1], english_tokenizer).replace('<PAD>', ''))
    prediction = model.predict(tmp_x[i:i+1])
    print("french : ", logits_to_text(prediction[0], french_tokenizer).replace('<PAD>', ''))
    print("\n")

english:  new jersey is sometimes quiet during autumn and it is snowy in april         
french :  new jersey est parfois calme pendant l' automne et il est neigeux en avril       


english:  the united states is usually chilly during july and it is usually freezing in november       
french :  les états unis est généralement froid en juillet et il gèle habituellement en novembre       


english:  california is usually quiet during march and it is usually hot in june         
french :  california est généralement calme en mars et il est généralement chaud en juin        


english:  the united states is sometimes mild during june and it is cold in september        
french :  les états unis est parfois légère en juin et il fait froid en septembre       


english:  your least liked fruit is the grape but my least liked is the apple        
french :  votre fruit aimé fruit est le raisin mais mon moins aimé est la pomme       


english:  his favorite fruit is the orange but my favorite 

french :  paris est parfois chaud en juin mais il est généralement chaud en juillet        


english:  paris is never hot during summer and it is usually mild in winter         
french :  paris est jamais chaude pendant l' été et il est généralement doux en hiver       


english:  she disliked a rusty yellow car                
french :  elle n'aimait pas une voiture jaune rouillée              


english:  france is usually quiet during november but it is sometimes warm in february         
french :  la france est généralement calme en novembre mais il est parfois chaud en février       


english:  new jersey is never wet during november and it is mild in august         
french :  new jersey est jamais humide au mois de novembre et il est doux au mois d' août    


english:  we like peaches pears and strawberries                
french :  nous aimons les pêches les poires et les fraises            


english:  the orange is her least liked fruit but the grapefruit is their least li

french :  elle conduit le vieux camion jaune               


english:  we like peaches mangoes and oranges                
french :  nous aimons les pêches les mangues et les oranges            


english:  new jersey is usually quiet during fall but it is usually warm in april        
french :  new jersey est généralement calme au cours de l' automne mais il est généralement chaud en avril    


english:  california is never rainy during winter and it is usually mild in summer         
french :  california est jamais pluvieux pendant l' hiver et il est généralement doux en été       


english:  california is sometimes cold during winter but it is sometimes chilly in autumn         
french :  california est parfois froid pendant l' hiver mais il est parfois frisquet à l' automne      


english:  she likes lemons pears and oranges                
french :  elle aime les citrons les poires et les oranges            


english:  we like strawberries bananas and oranges                


french :  paris est jamais tranquille au printemps mais il est généralement chaud en mars        


english:  paris is rainy during winter and it is sometimes warm in april          
french :  paris est pluvieux pendant l' hiver et il est parfois chaud en avril        


english:  you like pears grapefruit and limes                
french :  vous aimez les poires les pamplemousses et citrons verts            


english:  paris is warm during january and it is rainy in october           
french :  paris est chaud en cours et il pleut il pleut           


english:  france is sometimes quiet during december and it is never snowy in january         
french :  la france est parfois calme en décembre et il est jamais de neige en janvier      


english:  paris is sometimes relaxing during summer but it is usually mild in november         
french :  paris est relaxant parfois pendant l' été mais il est généralement doux en novembre       


english:  she likes bananas and grapes             

french :  californie est parfois froid pendant l' hiver mais il est généralement pluvieux en juin       


english:  china is sometimes wet during august but it is never quiet in october         
french :  chine est parfois humide au mois d' août mais il est jamais tranquille en octobre      




## Prediction (IMPLEMENTATION)

In [40]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
    tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

    model = model_final(
        tmp_x.shape,
        max_french_sequence_length,
        english_vocab_size,
        french_vocab_size)
    model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

input_shape: (137861, 21)
output_sequence_length: 21
english_vocab_size: 199
french_vocab_size: 344


input_seq.shape: (None, 21)
embedding.shape: (None, 21, 256)
Epoch 1/10
862/862 [==============================] - 14s 17ms/step - loss: 1.3454 - accuracy: 0.6549 - val_loss: nan - val_accuracy: 0.7493
Epoch 2/10
862/862 [==============================] - 14s 16ms/step - loss: 0.6184 - accuracy: 0.8136 - val_loss: nan - val_accuracy: 0.8680
Epoch 3/10
862/862 [==============================] - 14s 16ms/step - loss: 0.3116 - accuracy: 0.9074 - val_loss: nan - val_accuracy: 0.9362
Epoch 4/10
862/862 [==============================] - 14s 17ms/step - loss: 0.1801 - accuracy: 0.9470 - val_loss: nan - val_accuracy: 0.9535
Epoch 5/10
862/862 [==============================] - 15s 17ms/step - loss: 0.1301 - accuracy: 0.9605 - val_loss: nan - val_accuracy: 0.9630
Epoch 6/10
862/862 [==============================] - 14s 17ms/step - loss: 0.1023 - accuracy: 0.9685 - val_loss: nan - val_accuracy

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [2]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?